In [35]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


### Let's prepare the environment

# Lab | Natural Language Processing
### SMS: SPAM or HAM

In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

- Read Data for the Fraudulent Email Kaggle Challenge
- Reduce the training set to speead up development. 

In [37]:
## Read Data for the Fraudulent Email Kaggle Challenge
data = pd.read_csv("../data/kg_train.csv",encoding='latin-1')

# Reduce the training set to speed up development. 
# Modify for final system
data = data.head(1000)
print(data.shape)
data.fillna("",inplace=True)

(1000, 2)


### Let's divide the training and test set into two partitions

In [38]:
train = data[:800]
test = data[800:]

## Data Preprocessing

In [39]:
import string
from nltk.corpus import stopwords
print(string.punctuation)
print(stopwords.words("english")[100:110])
from nltk.stem.snowball import SnowballStemmer
snowball = SnowballStemmer('english')

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
['needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on']


## Now, we have to clean the html code removing words

- First we remove inline JavaScript/CSS
- Then we remove html comments. This has to be done before removing regular tags since comments can contain '>' characters
- Next we can remove the remaining tags

In [40]:
# 1. Remove inline JavaScript/CSS
# This regex removes <style>...</style> and <script>...</script> blocks
train['text'] = train['text'].apply(lambda x: re.sub(r'<style[^>]*>.*?</style>', '', x, flags=re.DOTALL))
train['text'] = train['text'].apply(lambda x: re.sub(r'<script[^>]*>.*?</script>', '', x, flags=re.DOTALL))

# 2. Remove HTML comments.
train['text'] = train['text'].apply(lambda x: re.sub(r'<!--.*?-->', '', x, flags=re.DOTALL))

# 3. Next we can remove the remaining tags
train['text'] = train['text'].apply(lambda x: re.sub(r'<[^>]*>', '', x))

C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\3280669416.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text'] = train['text'].apply(lambda x: re.sub(r'<style[^>]*>.*?</style>', '', x, flags=re.DOTALL))
C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\3280669416.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text'] = train['text'].apply(lambda x: re.sub(r'<script[^>]*>.*?</script>', '', x, flags=re.DOTALL))
C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\3280669416.py:7: SettingW

In [41]:
train

,text,label
0,"DEAR SIR, STRICTLY A PRIVATE BUSINESS PROPOSAL...",1
1,Will do.,0
2,Nora--Cheryl has emailed dozens of memos about...,0
3,Dear Sir=2FMadam=2C I know that this proposal ...,1
4,fyi,0
...,...,...
795,Overall the speech is worth reading; if you li...,0
796,Call Sheet.docHrc - did jake or anyone discuss...,0
797,newmyer on behalf of Jackie NewmyerTuesday Sep...,0
798,Yes we will. Thank you Courtney,0


- Remove all the special characters
    
- Remove numbers
    
- Remove all single characters
 
- Remove single characters from the start

- Substitute multiple spaces with single space

- Remove prefixed 'b'

- Convert to Lowercase

In [42]:
# Convert to Lowercase (doing this first for consistency in other regexes)
train['text'] = train['text'].apply(lambda x: x.lower())

# Remove prefixed 'b' (e.g., b'text' or "b'text") - common in some data sources
train['text'] = train['text'].apply(lambda x: re.sub(r"b['|\"](.*?)['|\"]", r'\1', x))
train['text'] = train['text'].apply(lambda x: x.lstrip('b')) # Remove 'b' at the very start if not part of b'' or b""

# Remove all special characters (keep spaces for now)
train['text'] = train['text'].apply(lambda x: re.sub(f'[{re.escape(string.punctuation)}]', '', x))

# Remove numbers
train['text'] = train['text'].apply(lambda x: re.sub(r'\d+', '', x))

# Substitute multiple spaces with single space (doing this before single char removal for better results)
train['text'] = train['text'].apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# Remove all single characters (e.g., 'a', 'b', 'c' that are stand-alone words)
# This includes single characters from the start, middle, or end if they are isolated words
train['text'] = train['text'].apply(lambda x: ' '.join([word for word in x.split() if len(word) > 1]))

# Remove single characters from the start (if any survived, though previous step should cover most)
train['text'] = train['text'].apply(lambda x: re.sub(r'^\S\s', '', x)) # Matches single non-space char at start followed by space

C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\3114994058.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text'] = train['text'].apply(lambda x: x.lower())
C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\3114994058.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text'] = train['text'].apply(lambda x: re.sub(r"b['|\"](.*?)['|\"]", r'\1', x))
C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\3114994058.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from

## Now let's work on removing stopwords
Remove the stopwords.

In [43]:
stop_words = set(stopwords.words('english'))
train['text'] = train['text'].apply(lambda text: ' '.join([
    word for word in text.split() if word not in stop_words
]))

C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\723668470.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text'] = train['text'].apply(lambda text: ' '.join([


## Tame Your Text with Lemmatization
Break sentences into words, then use lemmatization to reduce them to their base form (e.g., "running" becomes "run"). See how this creates cleaner data for analysis!

In [44]:
lemmatizer = WordNetLemmatizer()
train['text'] = train['text'].apply(lambda text: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(text)]))

C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\1438717788.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['text'] = train['text'].apply(lambda text: ' '.join([lemmatizer.lemmatize(word) for word in word_tokenize(text)]))


## Bag Of Words
Let's get the 10 top words in ham and spam messages (**EXPLORATORY DATA ANALYSIS**)

In [45]:
top_n = 10

# Top 10 words in ham messages
ham_words = ' '.join(train[train['label'] == 0]['text']).split()
ham_top = pd.Series(ham_words).value_counts().head(top_n)
print("Top 10 words in ham messages:")
print(ham_top)

# Top 10 words in spam messages
spam_words = ' '.join(train[train['label'] ==1]['text']).split()
spam_top = pd.Series(spam_words).value_counts().head(top_n)
print("\nTop 10 words in spam messages:")
print(spam_top)

Top 10 words in ham messages:
would        96
u            94
pm           88
president    84
state        81
call         75
time         71
mr           66
one          61
obama        56
Name: count, dtype: int64

Top 10 words in spam messages:
money          748
account        656
bank           638
fund           553
u              459
business       385
transaction    340
transfer       325
country        319
million        311
Name: count, dtype: int64


## Extra features

In [46]:
# We add to the original dataframe two additional indicators (money symbols and suspicious words).
money_simbol_list = "|".join(["euro","dollar","pound","€",r"\$"])
suspicious_words = "|".join(["free","cheap","sex","money","account","bank","fund","transfer","transaction","win","deposit","password"])

train['money_mark'] = train['text'].str.contains(money_simbol_list)*1
train['suspicious_words'] = train['text'].str.contains(suspicious_words)*1
train['text_len'] = train['text'].apply(lambda x: len(x)) 

test['money_mark'] = test['text'].str.contains(money_simbol_list)*1
test['suspicious_words'] = test['text'].str.contains(suspicious_words)*1
test['text_len'] = test['text'].apply(lambda x: len(x)) 

train.head()

C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\3284025728.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['money_mark'] = train['text'].str.contains(money_simbol_list)*1
C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\3284025728.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['suspicious_words'] = train['text'].str.contains(suspicious_words)*1
C:\Users\nicol\AppData\Local\Temp\ipykernel_1728\3284025728.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

,text,label,money_mark,suspicious_words,text_len
0,dear sir strictly private business proposal mi...,1,1,1,1505
1,,0,0,0,0
2,noracheryl emailed dozen memo haiti weekend pl...,0,0,0,110
3,dear sirfmadamc know proposal might surprise e...,1,1,1,1382
4,fyi,0,0,0,3


In [47]:
print("Original row 1:", data.iloc[1]['text'])
print("Processed row 1:", train.iloc[1]['text'])

Original row 1: Will do.
Processed row 1: 


In [48]:
train = train[train['text'].str.strip() != ""]

## How would work the Bag of Words with Count Vectorizer concept?

In [49]:
emails_text = train["text"].tolist()

bow_vectorizer = CountVectorizer()
X_bow = bow_vectorizer.fit_transform(emails_text)

## TF-IDF

- Load the vectorizer

- Vectorize all dataset

- print the shape of the vetorized dataset

In [50]:
tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(emails_text)

print(f"Vocabulary size (BoW): {len(bow_vectorizer.get_feature_names_out())}")
print(f"Vocabulary size (TF-IDF): {len(tfidf_vectorizer.get_feature_names_out())}")
print(f"Sample vocabulary (first 10): {list(bow_vectorizer.get_feature_names_out()[:10])}")

Vocabulary size (BoW): 16438
Vocabulary size (TF-IDF): 16438
Sample vocabulary (first 10): ['aac', 'aaclocated', 'aae', 'aaronovitchon', 'abacha', 'abachabefore', 'abachac', 'abachace', 'abachaco', 'abachacthe']


## And the Train a Classifier?

In [51]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# X_bow is your Bag of Words features, train['label'] is your target
y_train = train['label']

# Train RandomForest
#rf = RandomForestClassifier(random_state=42)
nb_bow = MultinomialNB()
#rf.fit(X_bow, y_train)
nb_bow.fit(X_bow, y_train)

# Predict on training data (for demonstration)
#y_pred = rf.predict(X_bow)
y_pred = nb_bow.predict(X_bow)
print("Training accuracy:", accuracy_score(y_train, y_pred))

Training accuracy: 0.9720101781170484


In [52]:
test = test[test['text'].str.strip() != ""]

# Transform test set using the same vectorizer
X_test_bow = bow_vectorizer.transform(test['text'].tolist())

# Predict and evaluate
y_test = test['label']
y_test_pred = nb_bow.predict(X_test_bow)
print("Test accuracy:", accuracy_score(y_test, y_test_pred))

Test accuracy: 0.905


### Extra Task - Implement a SPAM/HAM classifier

https://www.kaggle.com/t/b384e34013d54d238490103bc3c360ce

The classifier can not be changed!!! It must be the MultinimialNB with default parameters!

Your task is to **find the most relevant features**.

For example, you can test the following options and check which of them performs better:
- Using "Bag of Words" only
- Using "TF-IDF" only
- Bag of Words + extra flags (money_mark, suspicious_words, text_len)
- TF-IDF + extra flags


You can work with teams of two persons (recommended).

In [53]:

from scipy.sparse import hstack

# 1. Bag of Words only
nb_bow.fit(X_bow, y_train)
y_pred_bow = nb_bow.predict(X_test_bow)
print("Bag of Words only - Test accuracy:", accuracy_score(y_test, y_pred_bow))

# 2. TF-IDF only
X_test_tfidf = tfidf_vectorizer.transform(test['text'].tolist())
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_tfidf, y_train)
y_pred_tfidf = nb_tfidf.predict(X_test_tfidf)
print("TF-IDF only - Test accuracy:", accuracy_score(y_test, y_pred_tfidf))

# 3. Bag of Words + extra flags
extra_train = train[['money_mark', 'suspicious_words', 'text_len']].values
extra_test = test[['money_mark', 'suspicious_words', 'text_len']].values
X_bow_extra = hstack([X_bow, extra_train])
X_test_bow_extra = hstack([X_test_bow, extra_test])
nb_bow_extra = MultinomialNB()
nb_bow_extra.fit(X_bow_extra, y_train)
y_pred_bow_extra = nb_bow_extra.predict(X_test_bow_extra)
print("Bag of Words + extra flags - Test accuracy:", accuracy_score(y_test, y_pred_bow_extra))

# 4. TF-IDF + extra flags
X_tfidf_extra = hstack([X_tfidf, extra_train])
X_test_tfidf_extra = hstack([X_test_tfidf, extra_test])
nb_tfidf_extra = MultinomialNB()
nb_tfidf_extra.fit(X_tfidf_extra, y_train)
y_pred_tfidf_extra = nb_tfidf_extra.predict(X_test_tfidf_extra)
print("TF-IDF + extra flags - Test accuracy:", accuracy_score(y_test, y_pred_tfidf_extra))

Bag of Words only - Test accuracy: 0.905
TF-IDF only - Test accuracy: 0.94
Bag of Words + extra flags - Test accuracy: 0.87
TF-IDF + extra flags - Test accuracy: 0.6
